# Phantom silhouette法のデモ

## 必要な諸々のインポート

In [1]:
from typing import List
import librosa
import soundfile as sf
import pyworld as pw
import numpy as np
from PhantomSilhouette import *

In [2]:
import IPython.display
import librosa.display
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider
from scipy import interpolate

## 関数を定義

In [3]:
def pan(wav: List[np.ndarray], right_ratio: float = 0)  -> np.ndarray:
    """
    パン
    
    Parameters
    ----------
    wav: np.ndarray
        音声波形
    right_ratio: float
        右の音量の割合(-1~1, default 0)
    
    Returns
    ----------
    wav_out: np.ndarray
        パンした音声
    """
    wav[0] = wav[0] * (1 - right_ratio)
    wav[1] = wav[1] * (1 + right_ratio)
    return wav

In [4]:
def convert(wav_side: np.ndarray, sr: int) -> np.ndarray:
    """
    phantom silhouette法で音声の変換
    
    Parameters
    ----------
    wav_side: np.ndarray
        片方の音声
    sr: int
        サンプルレート
    
    Returns
    ----------
    wav_out: np.ndarray
        変換した音声
    """
    wav = wav_side.astype(np.float64)
    f0, sp, ap = pw.wav2world(wav, sr)
    f0_out, sp_out = phantom_silhouette(f0, sp, sr)
    wev_out = pw.synthesize(f0_out, sp_out, ap, sr)
    return wev_out.astype(np.float32)[:wav.shape[0]]

## 実際に変換してみる

In [5]:
wav, sr = librosa.load('suzuki.wav', mono=False)

In [6]:
IPython.display.Audio(wav, rate=sr)

In [7]:
wav[0] = convert(wav[0], sr)
wav[1] = convert(wav[1], sr)

In [8]:
wav = pan(wav, right_ratio=0)

In [9]:
IPython.display.Audio(wav, rate=sr)